# AWS DataZone의 REST API를 직접 호출하여 공유된 App 리스트 조회
- SMUS 공유된 App 화면의 경우 https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/listings/search 에 attribute:"typeName",value:"BedrockPromptAssetType" 페이로드를 이용해서 도메인 내 모든 BedrockPromptAssetType을 가져옴.

- 그리고, https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions 에 owningUserId:"2e2de543-52bd-4110-95c9-a8eda4a20ffb", status:"APPROVED" 페이로드를 이용해서 특정 User(owningUserId)에게 APPROVED된 subscription(사용자가 어떤 Listing을 구독하면, 시스템이 각 구독 요청마다 고유 ID를 생성)을 가져옴

- 두 가지를 조합하면 특정 사용자에게 승인된 BedrockPromptAssetType를 가져올 수 있음.

- 다만, https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/listings/search 의 경우 AWS Python SDK에서 search_listing 메서드를 사용해서 가져 올 수 있으나, https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions 이 API의 경우 AWS Python SDK에 특정 User(owningUserId) 기준으로 가져 올 수 있는 메서드를 제공하고 있지 않음.

- 결론적으로 https://datazone.ap-northeast-2.api.aws/v2/domains/dzd-cjvglgj4d43fmg/subscriptions API를 조회 후 필터링하여 BedrockPromptAssetType 만 조회하도록 구현

### 공통 설정

In [1]:
import boto3
import json
from requests_aws4auth import AWS4Auth
import requests

region = "ap-northeast-2"
domain_id = "dzd-cjvglgj4d43fmg"
user_email = "jh.bae@sk.com"

# Boto3 세션 & 인증
session = boto3.Session()
credentials = session.get_credentials().get_frozen_credentials()

auth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    "datazone",
    session_token=credentials.token
)


### 특정 사용자의 정보(현 예시는 iam계정이름)를 기반으로 SMUS 사용자 ID(ex:user-xxx-xxxxxxx-xxxx-xxxx) 조회

In [2]:
client = boto3.client("datazone", region_name=region)

response = client.search_user_profiles(
    domainIdentifier=domain_id,
    userType="DATAZONE_IAM_USER",
    searchText=user_email
)

items = response.get("items", [])
if not items:
    raise ValueError(f"{user_email} 사용자를 찾을 수 없음")

user_id = items[0]["id"]
iam_arn = items[0]["details"]["iam"]["arn"]
status = items[0]["status"]

print(f"UserId: {user_id}\nIAM ARN: {iam_arn}\nStatus: {status}")


UserId: 2e2de543-52bd-4110-95c9-a8eda4a20ffb
IAM ARN: arn:aws:iam::533616270150:user/jh.bae@sk.com
Status: ACTIVATED


### 사용자 구독 Bedrock Chat Assets 조회

In [3]:
url = f"https://datazone.{region}.api.aws/v2/domains/{domain_id}/subscriptions"
params = {
    "owningUserId": user_id,
    "status": "APPROVED"
}

response = requests.get(url, auth=auth, params=params)
response.raise_for_status()
all_data = response.json()

# BedrockChatAssetType만 필터링
bedrock_chat_assets = [
    item for item in all_data.get("items", [])
    if item.get("subscribedListing", {}).get("item", {}).get("assetListing", {}).get("entityType") == "BedrockChatAssetType"
]

agent_mapping = {}
for asset in bedrock_chat_assets:
    listing_id = asset["subscribedListing"]["id"]
    listing_name = asset["subscribedListing"]["name"]
    agent_mapping[listing_name] = {"listing_id": listing_id}

print("✅ Bedrock Chat Assets Mapping:")
print(json.dumps(agent_mapping, indent=4))


✅ Bedrock Chat Assets Mapping:
{
    "dataassettestagent": {
        "listing_id": "6ik8dtf37ip9co"
    },
    "Untitled App - wvstb": {
        "listing_id": "cz0wnjp047y97c"
    }
}


### Alias ID & Environment ID 조회

In [4]:
for agent_name, info in agent_mapping.items():
    listing_id = info["listing_id"]
    url = f"https://datazone.{region}.api.aws/v2/domains/{domain_id}/listings/{listing_id}"
    response = requests.get(url, auth=auth)
    response.raise_for_status()
    data = response.json()

    forms_str = data["item"]["assetListing"].get("forms", "{}")
    forms = json.loads(forms_str)
    bedrock_form = forms.get("BedrockAppCommonForm", {})

    alias_id = bedrock_form.get("sharedAliasOrVersion")
    env_id = bedrock_form.get("environmentId")

    agent_mapping[agent_name]["alias_id"] = alias_id
    agent_mapping[agent_name]["environment_id"] = env_id

print("✅ Alias & Environment Mapping:")
print(json.dumps(agent_mapping, indent=4))


✅ Alias & Environment Mapping:
{
    "dataassettestagent": {
        "listing_id": "6ik8dtf37ip9co",
        "alias_id": "NL5VG2E22Y",
        "environment_id": "cfpk72y5uyr7dk"
    },
    "Untitled App - wvstb": {
        "listing_id": "cz0wnjp047y97c",
        "alias_id": "N8ZYB1DCST",
        "environment_id": "619j2m0x33e848"
    }
}


### SMUS내에서 생성된 Bedrock Agent는 모두 Bedrock-Agent-{Environment ID} 형식으로 생성되며, 해당 이름을 기준으로  Bedrock Agent ID 조회하고 Alias ID까지 맵핑 (실제 Agent 호출 시 AgentID와 AliasID가 필요하기 때문)

In [5]:
bedrock_client = boto3.client("bedrock-agent", region_name=region)

# 전체 에이전트 조회
agents = []
next_token = None
while True:
    params = {"maxResults": 20}
    if next_token:
        params["nextToken"] = next_token

    resp = bedrock_client.list_agents(**params)
    agents.extend(resp.get("agentSummaries", []))
    next_token = resp.get("nextToken")
    if not next_token:
        break

# environment_id 기준으로 agent_id 매핑
for agent_name, info in agent_mapping.items():
    env_id = info.get("environment_id")
    expected_name = f"Bedrock-Agent-{env_id}"
    found_agent = next((a for a in agents if a["agentName"] == expected_name), None)
    if found_agent:
        agent_mapping[agent_name]["agent_id"] = found_agent["agentId"]
    else:
        agent_mapping[agent_name]["agent_id"] = None
        print(f"⚠️ {expected_name} 에 해당하는 에이전트 없음")

# 불필요한 environment_id 제거
for info in agent_mapping.values():
    info.pop("environment_id", None)

print("\n=== ✅ 최종 Agent Mapping ===")
print(json.dumps(agent_mapping, indent=4))



=== ✅ 최종 Agent Mapping ===
{
    "dataassettestagent": {
        "listing_id": "6ik8dtf37ip9co",
        "alias_id": "NL5VG2E22Y",
        "agent_id": "JKFP4XB9AO"
    },
    "Untitled App - wvstb": {
        "listing_id": "cz0wnjp047y97c",
        "alias_id": "N8ZYB1DCST",
        "agent_id": "0LPVTFQAEB"
    }
}
